In [ ]:
import polars as pl
import networkx as nx
from math import pi, sqrt
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path     = kagglehub.dataset_download("konivat/tree-of-life")
df_edges = None
df_nodes = None
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    if   'source_node_id' in df.columns and 'target_node_id' in df.columns: df_edges = df
    elif 'node_id'        in df.columns:                                    df_nodes = df
    else:                                                                   raise Exception("unknown file")

In [ ]:
src_col, dst_col = 'source_node_id', 'target_node_id'
_root_           = 1
_relates_        = [('source_node_id', 'target_node_id')]
def childrenOf(_node_):
    _df_ = df_edges.filter(pl.col(src_col) == _node_)
    return list(_df_[dst_col])
def totalLeaves(_node_, _levels_):
    if _levels_ == 0: return 1
    _sum_ = 0
    for _child_ in childrenOf(_node_): _sum_ += totalLeaves(_child_, _levels_-1)
    return _sum_

totalLeaves(_root_, 1)

In [ ]:
def renderHierarchy(node, levels=3, into_circle=None):
    if into_circle is None: into_circle = (0.0, 0.0, 1.0) # unit circle
    if     levels == 1:
        n = totalLeaves(node, 1)
        _circles_ = []
        for i in range(n): _circles_.append((0.0, 0.0, 1.0))
        _packed_         = rt.packCircles(_circles_, into_circle=into_circle)
        _node_to_circle_ = {}
        for _child_ in childrenOf(node): _node_to_circle_[_child_] = _packed_.pop(0)
        return _node_to_circle_
    else:
        area                = pi * into_circle[2] ** 2
        n                   = totalLeaves(node, levels)
        _circles_           = []
        _children_          = childrenOf(node)
        _radii_to_children_ = {}
        for _child_ in _children_:
            _leaf_count_        = totalLeaves(_child_, levels-1)
            _area_for_child_    = _leaf_count_ * area / n
            _child_radius_      = sqrt(_area_for_child_/pi)
            if _child_radius_ not in _radii_to_children_: _radii_to_children_[_child_radius_] = []
            _radii_to_children_[_child_radius_].append(_child_)
            _circles_.append((0.0, 0.0, _child_radius_))
        _packed_         = rt.packCircles(_circles_, into_circle=into_circle)
        # okay ... so this isn't going to work because we don't retain the order of the packed nodes through the packing algorithm...


renderHierarchy(_root_, 2)